In [1]:
import tensorflow as tf

/home/hahdawg/opt/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def temporal_convolution_layer(inputs, output_units, convolution_width, causal=False, dilation_rate=[1], bias=True,
                               activation=None, dropout=None, scope='temporal-convolution-layer', reuse=False):
    """
    Convolution over the temporal axis of sequence data.
    Args:
        inputs: Tensor of shape [batch size, max sequence length, input_units].
        output_units: Output channels for convolution.
        convolution_width: Number of timesteps to use in convolution.
        causal: Output at timestep t is a function of inputs at or before timestep t.
        dilation_rate:  Dilation rate along temporal axis.
    Returns:
        Tensor of shape [batch size, max sequence length, output_units].
    """
    with tf.variable_scope(scope, reuse=reuse):
        if causal:
            shift = int(convolution_width / 2) + int(int(dilation_rate[0] - 1) / 2)
            pad = tf.zeros([tf.shape(inputs)[0], shift, inputs.shape.as_list()[2]])
            inputs = tf.concat([pad, inputs], axis=1)

        W = tf.get_variable(
            name='weights',
            initializer=tf.random_normal_initializer(
                mean=0,
                stddev=1.0 / tf.sqrt(float(convolution_width)*float(shape(inputs, 2)))
            ),
            shape=[convolution_width, shape(inputs, 2), output_units]
        )

        z = tf.nn.convolution(inputs, W, padding='SAME', dilation_rate=dilation_rate)
        if bias:
            b = tf.get_variable(
                name='biases',
                initializer=tf.constant_initializer(),
                shape=[output_units]
            )
            z = z + b
        z = activation(z) if activation else z
        z = tf.nn.dropout(z, dropout) if dropout is not None else z
        z = z[:, :-shift, :] if causal else z
        return z


def time_distributed_dense_layer(inputs, output_units, bias=True, activation=None, batch_norm=None,
                                 dropout=None, scope='time-distributed-dense-layer', reuse=False):
    """
    Applies a shared dense layer to each timestep of a tensor of shape [batch_size, max_seq_len, input_units]
    to produce a tensor of shape [batch_size, max_seq_len, output_units].
    Args:
        inputs: Tensor of shape [batch size, max sequence length, ...].
        output_units: Number of output units.
        activation: activation function.
        dropout: dropout keep prob.
    Returns:
        Tensor of shape [batch size, max sequence length, output_units].
    """
    with tf.variable_scope(scope, reuse=reuse):
        W = tf.get_variable(
            name='weights',
            initializer=tf.random_normal_initializer(mean=0.0, stddev=1.0 / float(shape(inputs, -1))),
            shape=[shape(inputs, -1), output_units]
        )
        z = tf.einsum('ijk,kl->ijl', inputs, W)
        if bias:
            b = tf.get_variable(
                name='biases',
                initializer=tf.constant_initializer(),
                shape=[output_units]
            )
            z = z + b

        if batch_norm is not None:
            z = tf.layers.batch_normalization(z, training=batch_norm, reuse=reuse)

        z = activation(z) if activation else z
        z = tf.nn.dropout(z, dropout) if dropout is not None else z
        return z


def shape(tensor, dim=None):
    """Get tensor shape/dimension as list/int"""
    if dim is None:
        return tensor.shape.as_list()
    else:
        return tensor.shape.as_list()[dim]

In [3]:
def encode(x, features):
    x = tf.concat([x, features], axis=2)

    inputs = time_distributed_dense_layer(
        inputs=x,
        output_units=self.residual_channels,
        activation=tf.nn.tanh,
        scope='x-proj-encode'
    )

    skip_outputs = []
    conv_inputs = [inputs]
    for i, (dilation, filter_width) in enumerate(zip(self.dilations, self.filter_widths)):
        dilated_conv = temporal_convolution_layer(
            inputs=inputs,
            output_units=2*self.residual_channels,
            convolution_width=filter_width,
            causal=True,
            dilation_rate=[dilation],
            scope='dilated-conv-encode-{}'.format(i)
        )
        conv_filter, conv_gate = tf.split(dilated_conv, 2, axis=2)
        dilated_conv = tf.nn.tanh(conv_filter)*tf.nn.sigmoid(conv_gate)

        outputs = time_distributed_dense_layer(
            inputs=dilated_conv,
            output_units=self.skip_channels + self.residual_channels,
            scope='dilated-conv-proj-encode-{}'.format(i)
        )
        skips, residuals = tf.split(outputs, [self.skip_channels, self.residual_channels], axis=2)

        inputs += residuals
        conv_inputs.append(inputs)
        skip_outputs.append(skips)

    skip_outputs = tf.nn.relu(tf.concat(skip_outputs, axis=2))
    h = time_distributed_dense_layer(skip_outputs, 128, scope='dense-encode-1', activation=tf.nn.relu)
    y_hat = time_distributed_dense_layer(h, 1, scope='dense-encode-2')

    return y_hat, conv_inputs[:-1]


def initialize_decode_params(x, features):
    x = tf.concat([x, features], axis=2)

    inputs = time_distributed_dense_layer(
        inputs=x,
        output_units=self.residual_channels,
        activation=tf.nn.tanh,
        scope='x-proj-decode'
    )

    skip_outputs = []
    conv_inputs = [inputs]
    for i, (dilation, filter_width) in enumerate(zip(self.dilations, self.filter_widths)):
        dilated_conv = temporal_convolution_layer(
            inputs=inputs,
            output_units=2*self.residual_channels,
            convolution_width=filter_width,
            causal=True,
            dilation_rate=[dilation],
            scope='dilated-conv-decode-{}'.format(i)
        )
        conv_filter, conv_gate = tf.split(dilated_conv, 2, axis=2)
        dilated_conv = tf.nn.tanh(conv_filter)*tf.nn.sigmoid(conv_gate)

        outputs = time_distributed_dense_layer(
            inputs=dilated_conv,
            output_units=self.skip_channels + self.residual_channels,
            scope='dilated-conv-proj-decode-{}'.format(i)
        )
        skips, residuals = tf.split(outputs, [self.skip_channels, self.residual_channels], axis=2)

        inputs += residuals
        conv_inputs.append(inputs)
        skip_outputs.append(skips)

    skip_outputs = tf.nn.relu(tf.concat(skip_outputs, axis=2))
    h = time_distributed_dense_layer(skip_outputs, 128, scope='dense-decode-1', activation=tf.nn.relu)
    y_hat = time_distributed_dense_layer(h, 1, scope='dense-decode-2')
    return y_hat

In [4]:
class HelperClass(object):
    pass

self = HelperClass()

In [5]:
tf.reset_default_graph()
batch_size = 8
seq_len = 100
self.residual_channels = 32
self.skip_channels = 32
self.num_decode_steps = 10
self.decode_len = 64
num_features = 7
self.dilations = [2**i for i in range(8)]*3
self.filter_widths = [2 for i in range(8)]*3

x = tf.placeholder(tf.float32, shape=[batch_size, seq_len])
x = tf.expand_dims(x, 2)
self.encode_len = tf.placeholder(tf.int32, shape=[batch_size])
features = tf.placeholder(tf.float32, shape=[batch_size, seq_len, num_features])

y_hat_encode, conv_inputs = encode(x, features)
initialize_decode_params(x, features);

In [6]:
def print_shape(xs, name):
    print "{0}.shape = {1}".format(name, xs.shape)
    

In [30]:
batch_size = 8

cin = [[["c{0}{1}{2}".format(i, j, k) for k in xrange(3)]
       for j in xrange(200)]
       for i in xrange(batch_size)]
cin = np.array(cin)

dilation = 4

encode_len = 100 + np.arange(batch_size)
batch_idx = np.arange(batch_size)
batch_idx = np.tile(batch_idx[:, np.newaxis], (1, dilation))

qbt = encode_len - dilation - 1
temporal_idx = qbt[:, np.newaxis] + np.arange(dilation).reshape(1, -1)

idx = np.c_[batch_idx.reshape(-1, 1), temporal_idx.reshape(-1, 1)]

In [31]:
print batch_idx

[[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]
 [5 5 5 5]
 [6 6 6 6]
 [7 7 7 7]]


In [32]:
print temporal_idx

[[ 95  96  97  98]
 [ 96  97  98  99]
 [ 97  98  99 100]
 [ 98  99 100 101]
 [ 99 100 101 102]
 [100 101 102 103]
 [101 102 103 104]
 [102 103 104 105]]


In [35]:
idx[:10]

array([[ 0, 95],
       [ 0, 96],
       [ 0, 97],
       [ 0, 98],
       [ 1, 96],
       [ 1, 97],
       [ 1, 98],
       [ 1, 99],
       [ 2, 97],
       [ 2, 98]])

In [25]:
def print_shape(xs, name):
    print "{0}.shape = {1}".format(name, xs.shape)

# initialize state tensor arraysj
state_queues = []
for i, (conv_input, dilation) in enumerate(zip(conv_inputs, self.dilations)):
    
    print 75*"="
    print "dilation = {0}".format(dilation)
    print_shape(conv_input, "conv_input")
    
    batch_idx = tf.range(batch_size)
    batch_idx = tf.tile(tf.expand_dims(batch_idx, 1), (1, dilation))
    batch_idx = tf.reshape(batch_idx, [-1])
    print_shape(batch_idx, "batch_idx")
    
    queue_begin_time = self.encode_len - dilation - 1
    temporal_idx = tf.expand_dims(queue_begin_time, 1) + tf.expand_dims(tf.range(dilation), 0)
    temporal_idx = tf.reshape(temporal_idx, [-1])
    print_shape(temporal_idx, "temporal_idx")

    idx = tf.stack([batch_idx, temporal_idx], axis=1)
    print_shape(idx, "idx")
    slices = tf.gather_nd(conv_input, idx)
    print_shape(slices, "slices")
    slices = tf.reshape(slices, (batch_size, dilation, shape(conv_input, 2)))
    print_shape(slices, "slices")
    
    print "len(layer_ta) = {0}".format(dilation + self.num_decode_steps)
    layer_ta = tf.TensorArray(dtype=tf.float32, size=dilation + self.num_decode_steps)
    layer_ta = layer_ta.unstack(tf.transpose(slices, (1, 0, 2)))
    state_queues.append(layer_ta)

# initialize feature tensor array
features_ta = tf.TensorArray(dtype=tf.float32, size=self.num_decode_steps)
features_ta = features_ta.unstack(tf.transpose(features, (1, 0, 2)))

# initialize output tensor array
emit_ta = tf.TensorArray(size=self.num_decode_steps, dtype=tf.float32)

# initialize other loop vars
elements_finished = 0 >= self.decode_len
time = tf.constant(0, dtype=tf.int32)

# get initial x input
current_idx = tf.stack([tf.range(tf.shape(self.encode_len)[0]), self.encode_len - 1], axis=1)
initial_input = tf.gather_nd(x, current_idx)

dilation = 1
conv_input.shape = (8, 100, 32)
batch_idx.shape = (8,)
temporal_idx.shape = (8,)
idx.shape = (8, 2)
slices.shape = (8, 32)
slices.shape = (8, 1, 32)
len(layer_ta) = 11
dilation = 2
conv_input.shape = (8, 100, 32)
batch_idx.shape = (16,)
temporal_idx.shape = (16,)
idx.shape = (16, 2)
slices.shape = (16, 32)
slices.shape = (8, 2, 32)
len(layer_ta) = 12
dilation = 4
conv_input.shape = (8, 100, 32)
batch_idx.shape = (32,)
temporal_idx.shape = (32,)
idx.shape = (32, 2)
slices.shape = (32, 32)
slices.shape = (8, 4, 32)
len(layer_ta) = 14
dilation = 8
conv_input.shape = (8, 100, 32)
batch_idx.shape = (64,)
temporal_idx.shape = (64,)
idx.shape = (64, 2)
slices.shape = (64, 32)
slices.shape = (8, 8, 32)
len(layer_ta) = 18
dilation = 16
conv_input.shape = (8, 100, 32)
batch_idx.shape = (128,)
temporal_idx.shape = (128,)
idx.shape = (128, 2)
slices.shape = (128, 32)
slices.shape = (8, 16, 32)
len(layer_ta) = 26
dilation = 32
conv_input.shape = (8, 100, 32)
batch_idx.shape = (256,)
te

In [9]:
layer_ta.read(0).shape

TensorShape([Dimension(8), Dimension(32)])

In [26]:
skip_outputs, updated_queues = [], []
i = 0 
time = 0
current_input = initial_input
queues = state_queues
conv_input, queue, dilation = zip(conv_inputs, queues, self.dilations)[3]

current_features = features_ta.read(time)
current_input = tf.concat([current_input, current_features], axis=1)

with tf.variable_scope('x-proj-decode', reuse=True):
    w_x_proj = tf.get_variable('weights')
    b_x_proj = tf.get_variable('biases')
    x_proj = tf.nn.tanh(tf.matmul(current_input, w_x_proj) + b_x_proj)

state = queue.read(time)
with tf.variable_scope('dilated-conv-decode-{}'.format(i), reuse=True):
    w_conv = tf.get_variable('weights'.format(i))
    b_conv = tf.get_variable('biases'.format(i))
    dilated_conv = tf.matmul(state, w_conv[0, :, :]) + tf.matmul(x_proj, w_conv[1, :, :]) + b_conv
conv_filter, conv_gate = tf.split(dilated_conv, 2, axis=1)
dilated_conv = tf.nn.tanh(conv_filter)*tf.nn.sigmoid(conv_gate)

with tf.variable_scope('dilated-conv-proj-decode-{}'.format(i), reuse=True):
    w_proj = tf.get_variable('weights'.format(i))
    b_proj = tf.get_variable('biases'.format(i))
    concat_outputs = tf.matmul(dilated_conv, w_proj) + b_proj
skips, residuals = tf.split(concat_outputs, [self.skip_channels, self.residual_channels], axis=1)

x_proj += residuals
skip_outputs.append(skips)
updated_queues.append(queue.write(time + dilation, x_proj))


In [28]:
time

0

In [12]:
batch_size = 8
rchan = 1
seq_len = 30
dilation = 4



In [13]:
con_in = np.arange(batch_size*seq_len*rchan).reshape(batch_size, seq_len, rchan)
con_in = np.array([["enc_out[{0},{1}]".format(i, j) for j in xrange(seq_len)] for i in xrange(batch_size)])
pd.DataFrame(con_in)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,"enc_out[0,0]","enc_out[0,1]","enc_out[0,2]","enc_out[0,3]","enc_out[0,4]","enc_out[0,5]","enc_out[0,6]","enc_out[0,7]","enc_out[0,8]","enc_out[0,9]","enc_out[0,10]","enc_out[0,11]","enc_out[0,12]","enc_out[0,13]","enc_out[0,14]","enc_out[0,15]","enc_out[0,16]","enc_out[0,17]","enc_out[0,18]","enc_out[0,19]","enc_out[0,20]","enc_out[0,21]","enc_out[0,22]","enc_out[0,23]","enc_out[0,24]","enc_out[0,25]","enc_out[0,26]","enc_out[0,27]","enc_out[0,28]","enc_out[0,29]"
1,"enc_out[1,0]","enc_out[1,1]","enc_out[1,2]","enc_out[1,3]","enc_out[1,4]","enc_out[1,5]","enc_out[1,6]","enc_out[1,7]","enc_out[1,8]","enc_out[1,9]","enc_out[1,10]","enc_out[1,11]","enc_out[1,12]","enc_out[1,13]","enc_out[1,14]","enc_out[1,15]","enc_out[1,16]","enc_out[1,17]","enc_out[1,18]","enc_out[1,19]","enc_out[1,20]","enc_out[1,21]","enc_out[1,22]","enc_out[1,23]","enc_out[1,24]","enc_out[1,25]","enc_out[1,26]","enc_out[1,27]","enc_out[1,28]","enc_out[1,29]"
2,"enc_out[2,0]","enc_out[2,1]","enc_out[2,2]","enc_out[2,3]","enc_out[2,4]","enc_out[2,5]","enc_out[2,6]","enc_out[2,7]","enc_out[2,8]","enc_out[2,9]","enc_out[2,10]","enc_out[2,11]","enc_out[2,12]","enc_out[2,13]","enc_out[2,14]","enc_out[2,15]","enc_out[2,16]","enc_out[2,17]","enc_out[2,18]","enc_out[2,19]","enc_out[2,20]","enc_out[2,21]","enc_out[2,22]","enc_out[2,23]","enc_out[2,24]","enc_out[2,25]","enc_out[2,26]","enc_out[2,27]","enc_out[2,28]","enc_out[2,29]"
3,"enc_out[3,0]","enc_out[3,1]","enc_out[3,2]","enc_out[3,3]","enc_out[3,4]","enc_out[3,5]","enc_out[3,6]","enc_out[3,7]","enc_out[3,8]","enc_out[3,9]","enc_out[3,10]","enc_out[3,11]","enc_out[3,12]","enc_out[3,13]","enc_out[3,14]","enc_out[3,15]","enc_out[3,16]","enc_out[3,17]","enc_out[3,18]","enc_out[3,19]","enc_out[3,20]","enc_out[3,21]","enc_out[3,22]","enc_out[3,23]","enc_out[3,24]","enc_out[3,25]","enc_out[3,26]","enc_out[3,27]","enc_out[3,28]","enc_out[3,29]"
4,"enc_out[4,0]","enc_out[4,1]","enc_out[4,2]","enc_out[4,3]","enc_out[4,4]","enc_out[4,5]","enc_out[4,6]","enc_out[4,7]","enc_out[4,8]","enc_out[4,9]","enc_out[4,10]","enc_out[4,11]","enc_out[4,12]","enc_out[4,13]","enc_out[4,14]","enc_out[4,15]","enc_out[4,16]","enc_out[4,17]","enc_out[4,18]","enc_out[4,19]","enc_out[4,20]","enc_out[4,21]","enc_out[4,22]","enc_out[4,23]","enc_out[4,24]","enc_out[4,25]","enc_out[4,26]","enc_out[4,27]","enc_out[4,28]","enc_out[4,29]"
5,"enc_out[5,0]","enc_out[5,1]","enc_out[5,2]","enc_out[5,3]","enc_out[5,4]","enc_out[5,5]","enc_out[5,6]","enc_out[5,7]","enc_out[5,8]","enc_out[5,9]","enc_out[5,10]","enc_out[5,11]","enc_out[5,12]","enc_out[5,13]","enc_out[5,14]","enc_out[5,15]","enc_out[5,16]","enc_out[5,17]","enc_out[5,18]","enc_out[5,19]","enc_out[5,20]","enc_out[5,21]","enc_out[5,22]","enc_out[5,23]","enc_out[5,24]","enc_out[5,25]","enc_out[5,26]","enc_out[5,27]","enc_out[5,28]","enc_out[5,29]"
6,"enc_out[6,0]","enc_out[6,1]","enc_out[6,2]","enc_out[6,3]","enc_out[6,4]","enc_out[6,5]","enc_out[6,6]","enc_out[6,7]","enc_out[6,8]","enc_out[6,9]","enc_out[6,10]","enc_out[6,11]","enc_out[6,12]","enc_out[6,13]","enc_out[6,14]","enc_out[6,15]","enc_out[6,16]","enc_out[6,17]","enc_out[6,18]","enc_out[6,19]","enc_out[6,20]","enc_out[6,21]","enc_out[6,22]","enc_out[6,23]","enc_out[6,24]","enc_out[6,25]","enc_out[6,26]","enc_out[6,27]","enc_out[6,28]","enc_out[6,29]"
7,"enc_out[7,0]","enc_out[7,1]","enc_out[7,2]","enc_out[7,3]","enc_out[7,4]","enc_out[7,5]","enc_out[7,6]","enc_out[7,7]","enc_out[7,8]","enc_out[7,9]","enc_out[7,10]","enc_out[7,11]","enc_out[7,12]","enc_out[7,13]","enc_out[7,14]","enc_out[7,15]","enc_out[7,16]","enc_out[7,17]","enc_out[7,18]","enc_out[7,19]","enc_out[7,20]","enc_out[7,21]","enc_out[7,22]","enc_out[7,23]","enc_out[7,24]","enc_out[7,25]","enc_out[7,26]","enc_out[7,27]","enc_out[7,28]","enc_out[7,29]"


In [14]:
bi = np.arange(batch_size)
bi = np.tile(bi.reshape(-1, 1), (1, dilation))
bi = bi.reshape(-1, 1)

In [15]:
el = 10 + np.arange(batch_size)
qbt = el - dilation - 1
ti = qbt.reshape(-1, 1) + np.arange(dilation)
ti = ti.reshape(-1, 1)

In [16]:
idx = np.c_[bi, ti]

In [17]:
slices = np.array([con_in[i[0], i[1]] for i in idx])

In [18]:
slices = slices.reshape(batch_size, dilation)

In [19]:
slices

array([['enc_out[0,5]', 'enc_out[0,6]', 'enc_out[0,7]', 'enc_out[0,8]'],
       ['enc_out[1,6]', 'enc_out[1,7]', 'enc_out[1,8]', 'enc_out[1,9]'],
       ['enc_out[2,7]', 'enc_out[2,8]', 'enc_out[2,9]', 'enc_out[2,10]'],
       ['enc_out[3,8]', 'enc_out[3,9]', 'enc_out[3,10]', 'enc_out[3,11]'],
       ['enc_out[4,9]', 'enc_out[4,10]', 'enc_out[4,11]',
        'enc_out[4,12]'],
       ['enc_out[5,10]', 'enc_out[5,11]', 'enc_out[5,12]',
        'enc_out[5,13]'],
       ['enc_out[6,11]', 'enc_out[6,12]', 'enc_out[6,13]',
        'enc_out[6,14]'],
       ['enc_out[7,12]', 'enc_out[7,13]', 'enc_out[7,14]',
        'enc_out[7,15]']], dtype='|S13')

In [20]:
idx.shape

(32, 2)

In [21]:
np.take(np.arange(9).reshape(3, 3), [0, 0])

array([0, 0])

In [22]:
A = np.arange(9).reshape(3, 3)
A


array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [23]:
A[(1, 1)]

4